In [1]:
!pip install tensorflow

In [69]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import os
import time
import pickle

In [81]:
class Pipeline:
    
    def __init__(self):
        
        #dataset directory
        self.ROOT = pathlib.Path.cwd()/'PACKAGE'
        self.DATASET_DIR = self.ROOT/'DATASET'
        self.MODEL_DIR = self.ROOT/'MODEL'
        
        self.X_train = None
        self.X_test = None
        self.y_train = None
        self.y_test = None
        
    def create_directory(self):
        start = time.time()
        template = [self.ROOT, self.DATASET_DIR, self.MODEL_DIR]
        for directory in template:
            if directory.exists() == True:
                continue
            else:
                os.mkdir(directory)
        stop = time.time()
        total_time = stop - start
        print('Directory created, it took {0} s'.format(round(total_time, ndigits = 2)))
        del template
        
        

    def data_load(self):

        start = time.time()
        (self.X_train, self.y_train),(self.X_test, self.y_test) = tf.keras.datasets.fashion_mnist.load_data()
        stop = time.time()
        total_time = stop - start
        
        print('Data loaded, it took {0} s'.format(round(total_time, ndigits = 2)))
    
    def data_preprocess_and_save(self):
        start = time.time()
        
        self.X_train = self.X_train/255
        self.X_test = self.X_test/255
        self.y_train = tf.keras.utils.to_categorical(self.y_train)
        self.y_test = tf.keras.utils.to_categorical(self.y_test)
        
        with open(self.DATASET_DIR/'X_train.pkl', 'wb') as file:
            pickle.dump(self.X_train, file)
        with open(self.DATASET_DIR/'X_test.pkl', 'wb') as file:
            pickle.dump(self.X_test, file)
        with open(self.DATASET_DIR/'y_train.pkl', 'wb') as file:
            pickle.dump(self.y_train, file)
        with open(self.DATASET_DIR/'y_test.pkl', 'wb') as file:
            pickle.dump(self.y_test, file)
            
        stop = time.time()
        total_time = stop - start
        
        print('Data prepared and saved in directory, it took {0} s'.format(round(total_time, ndigits = 2)))
        
    def show_data(self, to_show = 32):
        fig = plt.figure(figsize = (12,12))
        
        for i in range(0, to_show):
            ax = fig.add_subplot(to_show/4, 4, i)
            ax.im
        
        
    def training_start(self):
        self.create_directory()
        self.data_load()
        self.data_preprocess_and_save()
    
        
        
        

In [82]:
obj1 = Pipeline()
obj1.training_start()

Directory created, it took 0.0 s
Data loaded, it took 1.48 s
I'm saving data
Data prepared and saved in directory, it took 4.11 s


In [41]:
obj1.MODEL_DIR

WindowsPath('A:/ML/OWN_PROJECT/14_fashion/PACKAGE/MODEL')

In [56]:
x = [i for i in range(0,10)]